In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

In [2]:
from tensorflow.keras.models import Sequential
import os

In [3]:

flowers_root = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

228813984/228813984 [==============================] - 1s 0us/step


In [4]:
flowers_root

'/root/.keras/datasets/flower_photos'

In [5]:
print( len(os.listdir('/root/.keras/datasets/flower_photos')))


6


In [9]:
print( len(os.listdir('/root/.keras/datasets/flower_photos/daisy')))
print( len(os.listdir('/root/.keras/datasets/flower_photos/dandelion')))
print( len(os.listdir('/root/.keras/datasets/flower_photos/roses')))
print( len(os.listdir('/root/.keras/datasets/flower_photos/sunflowers')))
print( len(os.listdir('/root/.keras/datasets/flower_photos/tulips')))


633
898
641
699
799


In [43]:
from keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        flowers_root,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

Found 3670 images belonging to 5 classes.


In [44]:
model = Sequential([
layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)),
layers.MaxPooling2D((2, 2)),
layers.MaxPooling2D((2, 2)),
layers.Conv2D(128, (3, 3), activation='relu'),
layers.MaxPooling2D((2, 2)),
layers.Conv2D(128, (3, 3), activation='relu'),
layers.MaxPooling2D((2, 2)),
layers.Flatten(),
layers.Dense(512, activation='relu'),
layers.Dense(5, activation='sigmoid')])

In [45]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20, 5)


In [46]:
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 74, 74, 32)        0         
 ng2D)                                                           
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 37, 37, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_10 (Conv2D)          (None, 35, 35, 128)       36992     
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 17, 17, 128)       0         
 ng2D)                                                           
                                                      

In [47]:
from tensorflow.keras.callbacks import LearningRateScheduler

In [48]:
def scheduler(epoch):
    if epoch < 10:
        return 0.001
    elif epoch < 15:
        return 0.0001
    else:
        return 0.00001

learning_rate_scheduler = LearningRateScheduler(scheduler, verbose=1)

In [49]:
from tensorflow.keras.callbacks import ModelCheckpoint


In [50]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [51]:
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch', period=3)

___________________________________________________

In [52]:
import datetime
from tensorflow.keras.callbacks import TensorBoard


In [53]:
logDir = ".\\fashion_mnist_log\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# logDir = "fashion_mnist_log/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") #If you are using linux
print("logDir: ", logDir)
tensorboard_callback = TensorBoard(logDir)

logDir:  .\fashion_mnist_log\20240723-202127


In [54]:
from keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer= optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [55]:
hh = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=20,
      callbacks=[learning_rate_scheduler,cp_callback,tensorboard_callback])


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/20


<ipython-input-55-d8cec73d6c25>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hh = model.fit_generator(


100/100 [==============================] - 5s 42ms/step - loss: 1.4213 - acc: 0.3779 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/20
100/100 [==============================] - 5s 51ms/step - loss: 1.1374 - acc: 0.5445 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/20
 99/100 [============================>.] - ETA: 0s - loss: 0.9950 - acc: 0.6111
Epoch 3: saving model to training_2/cp-0003.ckpt
100/100 [==============================] - 4s 44ms/step - loss: 1.0024 - acc: 0.6090 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.001.
Epoch 4/20
100/100 [==============================] - 5s 53ms/step - loss: 0.8703 - acc: 0.6470 - lr: 0.0010

Epoch 5: LearningRateScheduler setting learning rate to 0.001.
Epoch 5/20
100/100 [==============================] - 4s 41ms/step - loss: 0.8136 - acc: 0.6824 - lr: 0.0010

Epoch 6: LearningRateScheduler setting learning rate to 0.001.
Epoch 6/20
 99/10

# **It needs data validation to check overfitting, but we use its generation and callbacks**